# Meta analysis

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import functions as fun # functions mostly written by Alexendre Perez (and some by me)
from nistats import thresholding
import numpy as np
import nibabel as nib
import nibabel.processing
import nilearn
from nilearn import masking, plotting, image
from nilearn.input_data import NiftiMasker
import matplotlib.pyplot as plt
import matplotlib
from nimare.dataset import Dataset
import nimare
import shutil
import os
import scipy as sp


template = nilearn.datasets.load_mni152_template()
affine = template.affine

level = .05
height_control = 'fdr'
cluster_threshold = 1 # cluster-size threshold
cut_coords=(0, 0, 0) # brain coordinates for visualization

save_figs = False
save_results = False

hypothesis = 2

ModuleNotFoundError: No module named 'nipy'

Get the paths for the unthresholded image for each analysis team, for the given hypothesis.

In [ ]:
input_dir = '../data-narps/proc/'
img_paths = fun.get_data_paths_from_orig('hypo%d_unthresh.nii.gz' %hypothesis)

During testing, select the first N studies to speed things up. 

In [ ]:
# N_img = 5
# img_paths = img_paths[:N_img]

## Image-based meta-analysis

### Fixed-effects GLM

#### Get outcome variable (level-2 maps)

In [ ]:
order = 'C'
for n, path in enumerate(img_paths):
    img = nilearn.image.load_img(path)
    img_resampled = image.resample_to_img(img, template)
    array = img_resampled.get_fdata()
    flat = array.ravel(order=order)
    flat = np.array(flat)
    flat = np.expand_dims(flat, axis=1)
    if n == 0:
        y = flat
        voxel = np.argmax(flat)
    else:
        y = np.concatenate((y, flat), axis=1)
y=y.T

# z-score
#y = (y - np.nanmean(y, axis=0)) / np.nanstd(y, axis=0)

In [ ]:
font = {'size'   : 16}
matplotlib.rc('font', **font)

fig, ax = plt.subplots(figsize=(10,5))
plt.imshow(y[:, voxel: voxel+150])
ax.get_yaxis().set_ticks([])
ax.get_xaxis().set_ticks([])
plt.title('Y')
plt.xlabel('Voxels')
plt.ylabel('Level-2 maps\nfrom analysis teams')

if save_figs: plt.savefig('../figures/methods1_Y-in-glm.png')

#### Make design matrix

In [ ]:
x = np.ones_like(y[:,0])

In [ ]:
fig, ax = plt.subplots(figsize=(1,5))
plt.imshow(np.expand_dims(x, axis=1))
ax.get_yaxis().set_ticks([])
ax.get_xaxis().set_ticks([])
plt.title('X')
plt.xlabel(' ')
plt.ylabel('Level-2 maps\nfrom analysis teams')
if save_figs: plt.savefig('../figures/methods1_X-in-glm.png')

#### OLS

In [ ]:
Y = np.matrix(y)
X = np.matrix(x).T
Y = np.nan_to_num(Y, copy=False, nan=0)

betas = ((X.T * X)**-1) * X.T * Y
mean = np.mean(Y, axis=0)

#### Standard errors

In [ ]:
n = Y.shape[0]
residual = np.array(Y - X*betas)
SSE = np.squeeze(np.sum(np.power(residual,2), axis=0))
SD = np.sqrt(SSE/(n-1))
SE = SD/np.sqrt(n)

#### Z-map
Fisher's transformation from r-values to z-values 
(the betas are pearson's r values because there's only one predictor)

In [ ]:
zs = np.arctanh(betas)

#### T-map

In [ ]:
ts = betas/SE

In [ ]:
SE = fun.flat_mat_to_nii(flat_mat=SE, ref_niimg=img_resampled, order=order)
betas = fun.flat_mat_to_nii(flat_mat=betas, ref_niimg=img_resampled, order=order)
zs = fun.flat_mat_to_nii(flat_mat=zs, ref_niimg=img_resampled, order=order)
ts = fun.flat_mat_to_nii(flat_mat=ts, ref_niimg=img_resampled, order=order)
#mean = fun.flat_mat_to_nii(flat_mat=mean, ref_niimg=img_resampled, order=order)

In [ ]:
# thresholded
ts_thresh = thresholding.map_threshold(ts, alpha=level,
                                            height_control=height_control,
                                            cluster_threshold=cluster_threshold)[0]

## Visualize

In [ ]:
maps = { 
    'Level-3 beta map': betas,
    #'Level-3 z-map': zs,
    'Level-3 standard error map': SE,
    'Level-3 t-map': ts,
    'Level-3 t-map - thresholded': ts_thresh,
}
for name, img in maps.items():
    plotting.plot_stat_map(fun.mni_mask(img), title=name, cut_coords=cut_coords,
                          figure=plt.figure(figsize=(10,5)))
    
    if save_figs: plt.savefig('../figures/results1_hypo%d_%s.png'\
                                  %(hypothesis, name.replace(' ', '-')))
    if save_results: img.to_filename('../data-narps/results1_hypo%d_%s.nii.gz' \
                                  %(hypothesis, name.replace(' ', '-')))